In [1]:
import pandas as pd
import numpy as np
import random
import time


# Word Embedding
from sentence_transformers import SentenceTransformer

# Created functions
from Experimentations import ParamSearch
random.seed(0)

c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variables 

In [ ]:
vector_length = 768

# Data Import

In [ ]:
# Data import
# Relative Path of the dataset, change for your dataset
dataset_name = "cpn27"
file_path = ''

if dataset_name == "cpn27":
    # data = pd.read_csv(file_path+'CPN27_lemma.csv', delimiter=",")
    data = pd.read_csv(file_path+'CPN27_lemma.csv', delimiter=",")
    
elif dataset_name == "cpn120":
    data = pd.read_csv(file_path+'CPN120_lemma.csv', delimiter=",")

# else:
#     data = pd.read_csv(r'your-path/your-file.csv', delimiter=",")

data.fillna(value='')



,Palabra (concepto),Descripción,Codificación
0,Compasión,sentimiento,sentimiento
1,Compasión,por él que él él perdonar el vida a alguien en...,perdón
2,plan,necesario para cumplir objetivo,objetivos
3,plan,organizar recurso o persona,organización
4,plan,estrategia,estrategia
...,...,...,...
4933,Obligación,imposicion,imposición
4934,Obligación,carácter extricto,extricto
4935,Obligación,norma,normas
4936,Obligación,deber,deber


# Model

In [2]:
model = SentenceTransformer('intfloat/e5-base-v2')

In [ ]:
# Timer
start = time.time()

descriptions_matrix = np.zeros( # creatign an empty matrix
    (
        len(data),          # the number of data points
        vector_length       # the number of components of the word embedding
    )
)

# Matrix filling 
# Change to the name of the descriptions of your dataset.
for i,description in enumerate(data.iloc[:,1]):
    vector = model.encode(description)
    descriptions_matrix[i,] = vector

# Concatenate the matrix with the data of each observation
data_matrix = np.concatenate([descriptions_matrix,data], axis=1)


# Remove of the 'Nan' data
data_matrix = data_matrix[~pd.isnull(data_matrix[:,:vector_length]).any(axis=1)]


end = time.time()
print("Exceution time:", end-start)

Exceution time: 168.8829436302185


# AC-PLT

In [ ]:
from AC_PLT import AC_PLT

parameters = {'n_clusters': np.arange(50, 1800, 50)}
ac_plt = AC_PLT()

file_name = r'{}_{}_results.csv'.format(dataset_name, ac_plt.__class__.__name__)

clf_acplt = ParamSearch(ac_plt, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2],file_name)

# Naive Bayes

In [ ]:
from Experimentations import ParamSearch
from sklearn.naive_bayes import GaussianNB

parameters = {'var_smoothing': np.geomspace(1e-06, 1e+02, num=9)}
proc = GaussianNB()

file_name = r'{}_{}_results.csv'.format(dataset_name, proc.__class__.__name__)

clf = ParamSearch(proc, parameters)
clf.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2],file_name)

# SVC

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='rbf'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_rbf = SVC(kernel=kernel, gamma='auto')
fileName = r'{}_{}{}_results.csv'.format(dataset_name, svc_rbf.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_rbf, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='linear'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_linear = SVC(kernel=kernel, gamma='auto')
fileName = r'{}_{}{}_results.csv'.format(dataset_name, svc_linear.__class__.__name__, kernel)


clf_acplt = ParamSearch(svc_linear, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2])

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='poly'
parameters = {'degree': np.arange(1,7)}
svc_poly = SVC(kernel=kernel, gamma='auto')
fileName = r'{}_{}{}_results.csv'.format(dataset_name, svc_poly.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_poly, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='sigmoid'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_sigmoid = SVC(kernel=kernel, gamma='auto')
fileName = r'{}_{}{}_results.csv'.format(dataset_name, svc_rbf.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_sigmoid, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

# Decision Tree

In [ ]:
from Experimentations import ParamSearch
from sklearn.tree import DecisionTreeClassifier

criterion = 'gini'
parameters = {'max_leaf_nodes': np.arange(100, 1501, 100)}
desition_tree = DecisionTreeClassifier(criterion=criterion)
fileName = r'{}_{}_{}_results.csv'.format(dataset_name, desition_tree.__class__.__name__, criterion)

clf_acplt = ParamSearch(desition_tree, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

# XGBoost

In [ ]:
cod = data.iloc[:,vector_length+2].value_counts()
reduce_cod = cod[cod<5]
n=5
data_fill = data.copy()

for key, value in reduce_cod.items():
    m=np.abs(n-value)
    nrow = np.zeros(vector_length)
    nrow = np.concatenate([nrow, np.array(['', '', key])])
    for i in range(m): data_fill = np.vstack([data_fill,nrow])

In [ ]:
y = data_fill[:, vector_length+2]

labels = np.unique(y)
i=0
idx2class = {}
class2idx= {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1

In [ ]:
y_label = np.vectorize(class2idx.get)(y)

In [ ]:
max_leaves = [10,50,100,500,1000,1500,2000]

In [ ]:
from xgboost import XGBClassifier
from Experimentations import ParamSearch

parameters = {'max_leaves': max_leaves, 'n_estimators': np.arange(1, 20, 1)}

bst = XGBClassifier(learning_rate=1, objective='multi:softprob', random_state=0)
fileName = r'{}_{}_results.csv'.format(dataset_name, bst.__class__.__name__)

clf_acplt = ParamSearch(bst, parameters)
clf_acplt.fit(data_fill[:, :vector_length], y_label, fileName)


# KNN

In [ ]:
from Experimentations import ParamSearch
from sklearn.neighbors import KNeighborsClassifier

parameters = {'n_neighbors': np.arange(10, 501, 10)}
knn = KNeighborsClassifier()
fileName = r'{}_{}_results.csv'.format(dataset_name, knn.__class__.__name__)

clf_acplt = ParamSearch(knn, parameters)
clf_acplt.fit(data.iloc[:, :vector_length].to_numpy(), data.iloc[:, vector_length+2].to_numpy(), fileName)